In [ ]:
#  Working with weather data to plan future vacations

#  To do: Get countries and add to heat map
#  General clean up

In [1]:
# Dependencies
import gmaps
import gmaps.datasets
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests

from geopy.geocoders import Nominatim
from config import gkey

In [2]:
# Access maps with unique API key
gmaps.configure(api_key=gkey)

In [3]:
# Create a heat map that displays the humidity for every city from the part I 

# get cities from part I
cities_df = pd.read_csv("..\WeatherPy\WeatherPy_output.csv")

# capitalize city names - didn't do that in Part I
cities_df["City"] = cities_df.City.str.title()
cities_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Wind,Clouds
0,Ribeira Grande,38.52,-28.70,68.92,93,20.22,100
1,Yellowknife,62.46,-114.35,12.06,92,8.05,75
2,Westport,41.14,-73.36,31.51,80,5.82,1
3,Wajir,1.75,40.06,77.88,69,12.19,41
4,Rapid Valley,44.06,-103.15,55.76,49,14.99,1


In [4]:
# get parameters for heatmap
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()
locations = cities_df[["Latitude", "Longitude"]]


In [5]:
# generate heatmap
fig = gmaps.figure(center=(20,0),zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=maxhumidity,
                                 point_radius=3)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
# Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.

ideal_cities_df = (cities_df.loc[(cities_df["Temperature"] > 60) & (cities_df["Temperature"] < 80)  
              & (cities_df["Wind"] < 15) 
              & (cities_df["Clouds"] > 15) & (cities_df["Clouds"] < 35), :])


print(len(ideal_cities_df))
ideal_cities_df.head()


21


,City,Latitude,Longitude,Temperature,Humidity,Wind,Clouds
12,Hasaki,35.73,140.83,65.16,45,13.87,20
27,Guerrero Negro,27.98,-114.06,66.87,61,8.72,28
29,Merritt Island,28.54,-80.67,67.12,77,4.70,20
91,Victoria,22.29,114.16,73.47,62,4.00,31
135,Ohara,35.25,140.38,64.22,45,13.87,20


In [7]:
# reset the index
# was getting warnings from next set of code before adding this
# selecting subset preserved orginal index - guessing that screwed up the for-loop
ideal_cities_df = ideal_cities_df.dropna(how='any')
ideal_cities_df.reset_index(inplace=True)
del ideal_cities_df['index']
ideal_cities_df.head()

,City,Latitude,Longitude,Temperature,Humidity,Wind,Clouds
0,Hasaki,35.73,140.83,65.16,45,13.87,20
1,Guerrero Negro,27.98,-114.06,66.87,61,8.72,28
2,Merritt Island,28.54,-80.67,67.12,77,4.70,20
3,Victoria,22.29,114.16,73.47,62,4.00,31
4,Ohara,35.25,140.38,64.22,45,13.87,20


In [13]:
# get a hotel in each ideal city
# this takes a little while to execute

#setup
hotels = []
countries = []
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
geolocator = Nominatim(user_agent="RobertE_schoolwork")

#loop through ideal cities to get country and hotel
for index, row in ideal_cities_df.iterrows():
    lat = row['Latitude']
    lon = row['Longitude']
    params = {
        "location": f"{lat},{lon}",
        "radius": 5000,
        "types" : "hotel",
        "key": gkey
    }

    # get country from geopy
    geoloc_data = geolocator.reverse(f"{lat},{lon}",language="en")
    countries.append(geoloc_data.address.split(",")[-1])

    # get hotel from google
    request_output = requests.get(base_url, params=params)
    formatted_output = request_output.json()
    try:
        hotels.append(formatted_output['results'][1]['name'])
    except:
        hotels.append("No hotel found")

# add country and hotel data to ideal cities dataframe
ideal_cities_df["Hotel"] = hotels
ideal_cities_df["Country"] = countries
print(len(ideal_cities_df))
ideal_cities_df.head()

21


,City,Latitude,Longitude,Temperature,Humidity,Wind,Clouds,Hotel,Country
0,Hasaki,35.73,140.83,65.16,45,13.87,20,Inubosaki Kanko Hotel,Japan
1,Guerrero Negro,27.98,-114.06,66.87,61,8.72,28,Bimbo,Mexico
2,Merritt Island,28.54,-80.67,67.12,77,4.70,20,Prototype Development Laboratory,United States of America
3,Victoria,22.29,114.16,73.47,62,4.00,31,Four Seasons Hotel Hong Kong,China
4,Ohara,35.25,140.38,64.22,45,13.87,20,Isabaya,Japan


In [14]:
# Next cell disables scrollable frame in jupyter
# src: https://stackoverflow.com/questions/41641205/how-to-avoid-output-into-scrollable-frames-in-jupyter-notebook

# adding comment here because jupyter doesn't seem to like it when I add comments in same cell as %%javascript
# (all text on the # line turns red)
# to do: find out more about code in next cell

In [15]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [16]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
hotel_info = [info_box_template.format(**row) for index, row in ideal_cities_df.iterrows()]
locations = ideal_cities_df[["Latitude", "Longitude"]]
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
fig


Figure(layout=FigureLayout(height='420px'))